# Iterative Quantum Phase Estimation (IQPE) module

The present notebook reviews the **Iterative Quantum Phase Estimation (IQPE)** algorithm, which allows for the estimation of the phase of a unitary operator without using the **Quantum Fourier Transform (QFT)**. This **IQPE** algorithm has been implemented as the Python class `IQPE` within the module *iterative_quantum_pe* of the package **PE** in the current library `QQuantLib`(**QQuantLib/PE/iterative_quantum_pe**).

This notebook and its associated module are based on the following references:

- **Dobšíček, Miroslav**, **Johansson, Göran**, **Shumeiko, Vitaly**, and **Wendin, Göran**. *Arbitrary accuracy iterative quantum phase estimation algorithm using a single ancillary qubit: A two-qubit benchmark*. Physical Review A 76(3), 2007. [https://arxiv.org/abs/quant-ph/0610214](https://arxiv.org/abs/quant-ph/0610214)

- **Griffiths, Robert B.** and **Niu, Chi-Sheng**. *Semiclassical Fourier Transform for Quantum Computation*. Physical Review Letters 76(17), 1996. [https://arxiv.org/abs/quant-ph/9511007](https://arxiv.org/abs/quant-ph/9511007)

- **Kitaev, A. Y.**. *Quantum measurements and the Abelian stabilizer problem*. Electronic Colloquium on Computational Complexity, 3(3):1–22, 1996. [https://arxiv.org/abs/quant-ph/9511026](https://arxiv.org/abs/quant-ph/9511026)

- **Monz, Thomas**, **Nigg, Daniel**, **Martinez, Esteban A.**, **Brandl, Matthias F.**, **Schindler, Philipp**, **Rines, Richard**, **Wang, Shannon X.**, **Chuang, Isaac L.**, and **Blatt, Rainer**. *Realization of a scalable Shor algorithm*. Science 351(6277), 2016. [https://arxiv.org/abs/1507.08852](https://arxiv.org/abs/1507.08852)

In [ ]:
import sys
sys.path.append("../../")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm

In [ ]:
%matplotlib inline

In [ ]:
#This cell loads the QLM solver. See notebook: 00_AboutTheNotebooksAndQPUs.ipynb
from QQuantLib.qpu.get_qpu import get_qpu
# myqlm qpus: python, c
# QLM qpus accessed using Qaptiva Access library: qlmass_linalg, qlmass_mps
# QLM qpus: Only in local Quantum Learning Machine: linalg, mps
my_qpus = ["python", "c", "qlmass_linalg", "qlmass_mps", "linalg", "mps"]

linalg_qpu = get_qpu(my_qpus[1])

## 1. Example to use

To illustrate how the `IQPE` class works, we will use the **Iterative Quantum Phase Estimation (IQPE)** example from the Qiskit textbook:

- [Qiskit Tutorial: IQPE](https://github.com/Qiskit/qiskit-tutorials/blob/master/tutorials/algorithms/09_IQPE.ipynb)

This notebook contains 2 different examples:
- **IPE example with a 1-qubit gate for U**
- **IPE example with a 2-qubit gate**

The sections 2 and 3 will explain, step by step, how to use the `IQPE` class for solve the **IPE example with a 1-qubit gate for U** example. 

Section 4 will explain how to use the library solve the **IPE example with a 2-qubit gate**.

## 2. State and Unitary operator.

In the **IPE example with a 1-qubit gate for U** from the Qiskit textbook, the main problem is to find the phase for the 1-qubit unitary operator. They use the  $\mathcal{S}$ gate that has the following behaviour:

$$
\mathcal{S}|1\rangle = e^{i\frac{\pi}{2}}|1\rangle.
$$

This means that the $\mathcal{S}$ gate introduces a phase shift of $\frac{\pi}{2}$ radians to the state $|1\rangle$, leaving it unchanged otherwise.

To use the `IQPE` Python class from the module **QQuantLib/PE/iterative_quantum_pe**, two mandatory inputs must be provided:

1. **Initial State**: This is the initial quantum state required for applying the unitary operator ($|1\rangle$ in the Qiskit example). The initial state can be provided as:
    - A QLM QRoutine.
    - A QLM gate (or abstract gate).
2. **Unitary Operator**: This is the operator whose phase we aim to estimate ($\mathcal{S}$ gate in the Qiskit example). The unitary operator can be provided as:
    - A QLM QRoutine.
    - A QLM gate (or abstract gate).    



In [ ]:
# Initial quantum state
#Number Of Qubits
n_qbits = 1
initial_state = qlm.QRoutine()
q_bits = initial_state.new_wires(n_qbits)
for i in range(n_qbits):
    initial_state.apply(qlm.X, q_bits[i])
    
%qatdisplay initial_state --svg

In [ ]:
# unitary operator
unitary_operator = qlm.PH(np.pi/2.0)  
%qatdisplay unitary_operator --svg

## 3. Class `IQPE`: IQPE algorithm step by step  

The problem of phase estimation can be stated as follows. Given an initial state $|\Psi \rangle$ and a phase operator $\mathcal{P}$ such that:  
$$
\mathcal{P} |\Psi \rangle = e^{2\pi i \lambda} |\Psi \rangle,
$$  
our goal is to estimate $\lambda$.  

So far, we have the initial state $|\Psi \rangle = |1\rangle$ and the unitary operator whose phase we want to estimate, $\mathcal{P} = \mathcal{S}$. In this section, we will describe the class step by step and explain the basics of the **IQPE** algorithm.  

### 3.1 Calling the **IQPE** class  

The *IterativeQuantumPE* (`IQPE`) class is located in the **QQuantLib/PE/iterative_quantum_pe** module. To instantiate the class, you need to provide a Python dictionary. The mandatory keys are:  
* `initial_state`: A QLM routine or gate with the initial state $|\Psi \rangle$ loaded.  
* `unitary_operator`: A QLM gate or routine representing the Unitary operator to be applied to the initial state $|\Psi \rangle$.  

Other important keys include:  
* `cbits_number`: An integer specifying the number of classical bits needed for phase estimation.  
* `qpu`: A QLM solver. If not provided, the class attempts to create a `PyLinalg` solver. It is recommended to explicitly provide this key when instantiating the class.  
* `shots`: An integer specifying the number of shots for the quantum job.  

In [ ]:
#Load Class
from QQuantLib.PE.iterative_quantum_pe import IQPE

In [ ]:
n_cbits = 2
#We create a python dictionary for configuration of class
iqpe_dict = {
    'initial_state': initial_state,
    'unitary_operator': unitary_operator,
    'qpu' : linalg_qpu,
    'cbits_number' : n_cbits,
}
iqpe_object = IQPE(**iqpe_dict)

When the class is instantiated the properties `initial_state` and `q_gate` are overwritten with the given keys *initial_state* and *unitary_operator* respectively

In [ ]:
c = iqpe_object.initial_state
%qatdisplay c --svg

In [ ]:
a = iqpe_object.q_gate 
%qatdisplay a --depth 2 --svg

### 3.2 IPE Algorithm step by step

Now we are going to review step by step the **IQPE** algorithm using different programmed methods of the `IQPE` class.

#### 3.2.1 Initialize the quantum program

The first step is to call the method `init_iqpe`. This method performs the following actions:

1. Creation of a QLM program from the `initial_state` QLM routine (or `AbstractGate`). The QLM program is stored in the `q_prog` property.
2. Allocation of an auxiliary qubit, which is mandatory for the **IQPE** algorithm. This qubit is stored in the `q_aux` property.
3. Allocation of the auxiliary classical bits where the estimated phase will be stored. These bits are stored in the `c_bits` property.

In [ ]:
#Initialize the quantum program
iqpe_object.init_iqpe()

In [ ]:
#Now we have the initial quantum program stored in the property q_prog
#Additionally a auxiliary qubit bits was allocated
circuit = iqpe_object.q_prog.to_circ(submatrices_only=True)

%qatdisplay circuit --depth 0 --svg

#### 3.2.2 IQPE Algorithm

We are going to decompose the **IQPE** algorithm into two parts: a first part where the main variable $ l $ is 0 ($ l = 0 $), and a second recursive part where the variable $ l $ is greater than 0 ($ l > 0 $).

##### First Part ($ l = 0 $)

The first step of the **IQPE** algorithm ($ l = 0 $) consists of the following parts:

1. Reset the auxiliary qubit.
2. Apply a Hadamard gate to the auxiliary qubit.
3. Apply the phase operator $\mathcal{P}$, controlled by the auxiliary qubit, $2^{m-1}$ times ($\mathcal{P}^{2^{m-1}}$). Here, $ m $ is the number of classical bits allocated for estimating $\theta$.
4. Apply a Hadamard gate to the auxiliary qubit.
5. Measure the auxiliary qubit and store the result in a classical bit array at position $ c_l $ ($ l = 0 $).

This can be achieved by calling the `step_iqpe` method with the following arguments:
- Quantum Program with the `initial_state`.
- Quantum Routine or `AbstractGate` with the phase operator $\mathcal{P}$.
- Auxiliary Qubit.
- Auxiliary classical bits.
- $ l = 0 $.

The `step_iqpe` method returns the quantum program with the operations explained in this part.

In [ ]:
q_program = iqpe_object.q_prog
q_program = iqpe_object.step_iqpe(
    q_program, 
    iqpe_object.q_gate,
    iqpe_object.q_aux,
    iqpe_object.c_bits, 
    l=0
)

Following cell show the first  part of the circuit

In [ ]:
c = q_program.to_circ()
%qatdisplay c --svg

##### Second or Iterative Part ($ l > 0 $)

This part of the algorithm is recursive. The following steps will be repeated for each value of $ l = 1, 2, \dots, m-1 $ (where $ m $ is the number of classical bits used to encode the phase):

1. Reset the auxiliary qubit.
2. Apply a Hadamard gate to the auxiliary qubit.
3. Apply the phase operator $\mathcal{P}$, controlled by the auxiliary qubit, $ 2^{m-1-l} $ times ($ \mathcal{P}^{2^{m-1-l}} $). Here, $ m $ is the number of classical bits allocated for estimating $\theta$.
4. Apply a set of controlled rotations on the auxiliary qubit, conditioned on the classical bits $ c_j $, with angles:
   $$
   \frac{\pi}{2} \cdot \frac{1}{2^{l-j-1}}, \quad \text{for } j = 0, 1, \dots, l-1.
   $$
5. Apply a Hadamard gate to the auxiliary qubit.
6. Measure the auxiliary qubit and store the result in a classical bit array at position $ c_l $.

Thus, for each step $ l $, the controlled rotations depend on the measurements performed in the previous $ l $ steps.

In the following cell, we explain how to create the algorithm for the $ l = 1 $ part:

In [ ]:
#here we do the step l=1
l=1
q_program = iqpe_object.step_iqpe(
    q_program, 
    iqpe_object.q_gate, 
    iqpe_object.q_aux,
    iqpe_object.c_bits, 
    l=l)

Now we can plot the circuit we have until the moment

In [ ]:
#Circuit for l=0 and l=1
c = q_program.to_circ()
%qatdisplay c --svg

#### Complete Algorithm

For a complete **IQPE** algorithm, the following steps should be performed:

1. Create the first part of the algorithm for $ l = 0 $.
2. Iterate the second part of the algorithm from $ l = 1 $ to $ l = m-1 $, where $ m $ is the number of classical bits used for estimating the phase.

The measured classical bits are then used to estimate the phase of the unitary operator.

The following cell creates the complete program for the **IQPE** algorithm.

In [ ]:
#Initialize the quantum program
iqpe_object.init_iqpe()
q_program = iqpe_object.q_prog
for l in range(len(iqpe_object.c_bits)):
    q_program = iqpe_object.step_iqpe(
        q_program, 
        iqpe_object.q_gate,
        iqpe_object.q_aux,
        iqpe_object.c_bits,
        l=l)

So for the desired number of classical bits $m$ the complete circuit for *IPE* algorithm will be:

In [ ]:
c = q_program.to_circ()
%qatdisplay c --svg

#### 3.2.3 IQPE Algorithm Execution

Once the myQLM program is constructed, the algorithm should be executed. For this purpose, the `run_qprogram` method of the class allows you to execute it. The following arguments should be provided:

- `q_prog`: The complete QLM program containing the **IPE** algorithm.
- `q_aux`: The auxiliary qubit used in the algorithm.
- `shots`: The number of shots for the quantum job.
- `linalg_qpu`: The QLM solver (e.g., a linear algebra-based quantum processing unit).

This method creates the circuit, sets up the associated job, and executes it. The raw results of the simulation, along with the circuit, are returned.

In [ ]:
raw_results, circuit = iqpe_object.run_qprogram(
    q_program,
    iqpe_object.q_aux, 100, linalg_qpu=linalg_qpu)

In [ ]:
%qatdisplay circuit --svg

#### 3.2.4 IQPE: Getting Classical Bits Measurements

As explained, the phase is estimated by obtaining the measurements of the classical bits. In the class, this is achieved using the `measure_classical_bits` method. The input to this method is the `raw_results` obtained from the `run_qprogram` method. 

The output is a pandas DataFrame containing the aggregated measurements of the classical bits, with the following columns:

- `BitString`: The result of the classical bits measurement at each step of the algorithm.
- `BitInt`: The integer representation of the `BitString`.
- `Phi`: The estimated phase, computed as $\frac{\text{BitInt}}{2^{m}}$, where $m$ is the number of classical bits used for phase estimation.
- `Frequency`: The frequency of the obtained `BitString`, calculated as the number of times the `BitString` appears divided by the total number of `shots`.

In [ ]:
classical_bits = iqpe_object.measure_classical_bits(raw_results)

In [ ]:
classical_bits

The most important column fro the `classical_bits` DataFrame is the `Phi`one. Following the Qiskit example, this column corresponds to $\varphi$, and the searched phase is given by: 

$$
\phi = 2\pi\varphi.
$$ 
In the Qiskit example, $\varphi = 0.25$.

The `classical_bits` DataFrame provides the aggregated results for all executions of the circuit (determined by the variable `shots`).

#### 3.2.5 IQPE: Post-Processing

Typically, the phase in radians is provided in this type of phase estimation problem. To obtain this result, we use the `post_process` method. The `classical_bits` DataFrame should be provided as input, and the output will be another DataFrame with the following columns:

- `BitString`: The result of the classical bits measurement at each step of the algorithm.
- `BitInt`: The integer representation of the `BitString`.
- `Phi`: The estimated phase, computed as $\frac{\text{BitInt}}{2^{c_b}}$, where $c_b$ is the number of classical bits.
- `2*theta`: The phase of the unitary operator in radians, given by $2\theta = 2\pi\varphi$. In this case, the phase of the unitary operator is calculated as twice an angle $\theta$.
- `theta`: Half of the phase of the unitary operator, given by $\theta = \pi\varphi$.
- `theta_90`: The value of $\theta$ constrained to the interval $(0, \frac{\pi}{2})$.
- `Frequency`: The frequency of each `BitString`.

The returned DataFrame from `post_process` is ordered by decreasing values of the frequency, so the first row corresponds to the most frequent `BitString`, the second row to the second most frequent value, and so on.

In this module, our convention when $|\Psi\rangle$ is an eigenstate of a unitary operator $\mathcal{Q}$ is:
$$
\mathcal{Q}|\Psi\rangle = e^{2i\theta}|\Psi\rangle
$$

In [ ]:
final_results = iqpe_object.post_proccess(classical_bits)

In [ ]:
final_results

### 3.3 Class IQPE: Complete Execution

In Section 2.3, the complete algorithm was explained step by step. The methods described in that subsection were presented for pedagogical purposes only. **Users SHOULD NOT use these methods directly.** Instead, users **SHOULD USE** the `iqpe` method, which executes all the steps outlined in Section 2.2. 

When using this method, the following properties are populated:

- `classical_bits`: A DataFrame containing the results of the `measure_classical_bits` method.
- `final_results`: A DataFrame containing the results of the `post_process` method.

Additionally, the `iqpe` method updates the `pdf_time` property of the class, where several elapsed times are stored.

In [ ]:
n_cbits = 2
#We create a python dictionary for configuration of class
iqpe_dict = {
    'initial_state': initial_state,
    'unitary_operator': unitary_operator,
    'qpu' : linalg_qpu,
    'cbits_number' : n_cbits,  
    'shots': 100
}
iqpe_object = IQPE(**iqpe_dict)
iqpe_object.iqpe()

In [ ]:
#classical_bits attribute
iqpe_object.classical_bits

In [ ]:
#final_results attribute
iqpe_object.final_results

The attribute circuit of the class stores the myQLM circuit created

In [ ]:
c = iqpe_object.circuit
%qatdisplay c --svg

## 4.  IPE example with a 2-qubit gate

We will now reproduce the Qiskit example from the section **IPE example with a 2-qubit gate** (available in [Qiskit Tutorials](https://github.com/Qiskit/qiskit-tutorials/blob/master/tutorials/algorithms/09_IQPE.ipynb)).

In this scenario, the example uses **2 qubits** to estimate the phase of a unitary operator $\mathcal{cT}$. This operator introduces a phase shift of $\frac{\pi}{4}$ radians specifically to the state $|11\rangle$, leaving all other states unchanged. Mathematically, this behavior is expressed as:

$$
\mathcal{cT} |11\rangle = e^{i\frac{\pi}{4}} |11\rangle.
$$

From the phase estimation convention, we can derive the relationship between the phase shift and the parameter $\lambda$:

$$
i\frac{\pi}{4} = 2\pi i \lambda.
$$

Solving for $\lambda$, we obtain:

$$
\lambda = \frac{1}{8}.
$$

This value of $\lambda$ represents the desired solution for the phase estimation problem in this example.


In [ ]:
#create initial state
n_qbits = 2
initial_state = qlm.QRoutine()
q_bits = initial_state.new_wires(n_qbits)
for i in range(n_qbits):
    initial_state.apply(qlm.X, q_bits[i])

In [ ]:
%qatdisplay initial_state --svg

In [ ]:
#Create cT operator
unitary_operator = qlm.QRoutine()
uq_qbits = unitary_operator.new_wires(n_qbits)
unitary_operator.apply(qlm.PH(np.pi/4.0).ctrl(), 0, 1)

In [ ]:
%qatdisplay unitary_operator --svg

In [ ]:
#now IPE!!
n_cbits = 3
#We create a python dictionary for configuration of class
iqpe_dict = {
    'initial_state': initial_state,
    'unitary_operator': unitary_operator,
    'qpu' : linalg_qpu,
    'cbits_number' : n_cbits,  
    'shots': 100
}
iqpe_object = IQPE(**iqpe_dict)

In [ ]:
iqpe_object.iqpe()

In [ ]:
iqpe_object.classical_bits

The result for $\varphi=0.125$ that is the value obtained in the Qiskit example.

In [ ]:
c = iqpe_object.circuit
%qatdisplay c --svg

In [ ]:
iqpe_object.final_results

## 5. Application to Amplitude Estimation

The problem of amplitude estimation is the following. Given an oracle:

$$\mathcal{0}|0\rangle = |\Psi\rangle = \sqrt{a}|\Psi_0\rangle +\sqrt{1-a}|\Psi_1\rangle,$$

where $|\Psi_0\rangle$ and $|\Psi_1\rangle$ are orthogonal states, we want to estimate $\sqrt{a}$.




The *Iterative Quantum Phase Estimation* can be applied to *Amplitude Estimation* problem. The present section explains how we can take advantage of our **IQPE** class for this.

First, we will define the following amplitude estimation problem:
$$
    \begin{array}{l}
    & |\Psi\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    & \sqrt{a}|\Psi_0\rangle \longrightarrow \dfrac{\sqrt{1}}{\sqrt{0+1+2+3+4+5+6+7+8}}|1\rangle.\\
    & \sqrt{1-a}|\Psi_1\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    \end{array}
$$

The following cells create the initial state $|\Psi\rangle$

## 5. Application to Amplitude Estimation

The problem of **Amplitude Estimation** can be formulated as follows: Given an oracle operator $\mathcal{A}$ with the following behavior:

$$
|\Psi\rangle = \mathcal{A}|0\rangle = \sqrt{a}|\Psi_0\rangle + \sqrt{1-a}|\Psi_1\rangle,
$$

where $|\Psi_0\rangle$ and $|\Psi_1\rangle$ are orthogonal states, our goal is to estimate $\sqrt{a}$. 

The **IQPE** can be applied to **Amplitude Estimation** problem by doing the following steps:

1. **Substitution**: Replace $\sqrt{a}$ with $\sin(\theta)$:
   $$
   \sqrt{a} = \sin(\theta).
   $$

2. **Rewriting the Oracle Operator**: The action of the oracle operator becomes:
   $$
   |\Psi\rangle = \mathcal{A}|0\rangle = \sqrt{a}|\Psi_0\rangle + \sqrt{1-a}|\Psi_1\rangle = \sin(\theta)|\Psi_0\rangle + \cos(\theta)|\Psi_1\rangle. \tag{1}
   $$

3. **Creating the Grover-like Operator**: Construct the Grover-like operator for the oracle operator $\mathcal{A}$:
   $$
   \mathbf{G}(\mathcal{A}) = \mathcal{A} \left(\hat{I} - 2|0\rangle\langle 0|\right) \mathcal{A}^\dagger \left(\hat{I} - 2|\Psi_0\rangle\langle \Psi_0|\right).
   $$

With these substitutions, the **IQPE** algorithm can now be applied directly by replacing:

$$
\begin{aligned}
& |\Psi\rangle \longrightarrow \mathcal{A}|0\rangle, \\
& \mathcal{Q} \longrightarrow \mathbf{G}(\mathcal{A}).
\end{aligned}
$$

It can be demonstrated that the $\mathbf{G}(\mathcal{A})$ operator represents a rotation in the plane by an angle $2\theta$, expressed as:

$$
\mathbf{G}(\mathcal{A}) =
\begin{pmatrix}
\cos(2\theta) & -\sin(2\theta) \\
\sin(2\theta) & \cos(2\theta)
\end{pmatrix}.
$$

Thus, the eigenvalues of $\mathbf{G}(\mathcal{A})$ are given by $e^{\pm i2\theta}$.
Using the phase estimation convention:

$$
i2\theta = 2\pi i \lambda,
$$

we arrive at:

$$
\theta = \pi \lambda.
$$


### 5.1 Using `IQPE` class for *Amplitude Estimation*

In this section, we will demonstrate how to use the `IQPE` class to solve **Amplitude Estimation** problems.

#### Defining the Amplitude Estimation Problem

We begin by defining the following amplitude estimation problem:

$$
|\Psi\rangle = \mathcal{A}|0\rangle = \frac{1}{\sqrt{0+1+2+3+4+5+6+7+8}} \left[\sqrt{0}|0\rangle + \sqrt{1}|1\rangle + \sqrt{2}|2\rangle + \sqrt{3}|3\rangle + \sqrt{4}|4\rangle + \sqrt{5}|5\rangle + \sqrt{6}|6\rangle + \sqrt{7}|7\rangle \right]. \tag{2}
$$

By comparing equation (2) with the general form of the state in equation (1):

$$
\sqrt{a}|\Psi_0\rangle = \sin(\theta)|\Psi_0\rangle = \frac{\sqrt{1}}{\sqrt{0+1+2+3+4+5+6+7+8}}|1\rangle,
$$

and

$$
\sqrt{1-a}|\Psi_1\rangle = \cos(\theta)|\Psi_1\rangle = \frac{1}{\sqrt{0+1+2+3+4+5+6+7+8}} \left[\sqrt{0}|0\rangle + \sqrt{2}|2\rangle + \sqrt{3}|3\rangle + \sqrt{4}|4\rangle + \sqrt{5}|5\rangle + \sqrt{6}|6\rangle + \sqrt{7}|7\rangle \right].
$$

Here:
- The target state $|\Psi_0\rangle$ corresponds to $|1\rangle$, and its amplitude is proportional to $\sin(\theta)$.
- The remaining states collectively form $|\Psi_1\rangle$, with an amplitude proportional to $\cos(\theta)$.

#### Creating the Initial State

The subsequent cells will construct the initial state $|\Psi\rangle = \mathcal{A}|0\rangle$, which serves as the starting point for our amplitude estimation process. This involves loading the specified probability distribution into a quantum state using the appropriate functions from the library.

In [ ]:
from QQuantLib.DL.data_loading import load_probability
from QQuantLib.AA.amplitude_amplification import grover
from QQuantLib.PE.iterative_quantum_pe import IQPE

In [ ]:
n = 3
N = 2**n
x = np.arange(N)
probability = x/np.sum(x)
oracle = load_probability(probability)

%qatdisplay oracle --depth 0 --svg

Now we need to compute the Grover-like operator from the oracle operator $\mathcal{A}$. The functions from `AA` module will be used (see notebook **02_Amplitude_Amplification_Operators.ipynb** for more information)

In [ ]:
target = [0, 0, 1]
index = range(oracle.arity)
grover_gate = grover(oracle, target, index)
#Comment before line and uncomment following for For multiplexor implementation of multi-controlled Z gate
#grover_gate = grover(oracle, target, index, mcz_qlm=False)
%qatdisplay grover_gate --depth  --svg

Now that we have translated our amplitude amplification problem into a phase estimation problem, we can proceed to use our class in the usual way. We provide the `oracle` as the **initial_state** and the corresponding Grover-like operator as the **unitary_operator**. 

Additionally, the number of classical bits,  `cbits_number` required for estimating the phase should also be specified.

In [ ]:
n_cbits = 6
#We create a python dictionary for configuration of class
iqpe_dict = {
    'initial_state': oracle,
    'unitary_operator': grover_gate,
    'qpu' : linalg_qpu,
    'cbits_number' : n_cbits,  
    'shots': 100
}
iqpe_object = IQPE(**iqpe_dict)
iqpe_object.iqpe()

In [ ]:
iqpe_object.final_results

In [ ]:
#We can plot the obtained angles vs the its frequency
plt.plot(
    iqpe_object.final_results['theta_90'], 
    iqpe_object.final_results['Frequency'], 'o')
plt.xlabel(r'$\theta$')
plt.xlim(0, 0.5*np.pi)
plt.ylabel('Frecuency')
plt.title(r'$\theta \in [0, \frac{\pi}{2}]$')

Last, we use the mapping $a = \cos(\theta)^2$ to obtain the result of our amplitude estimation problem.

In [ ]:
theta_estimated = iqpe_object.final_results['theta_90'].iloc[0]
print('Quantum result, theta_estimated: ', theta_estimated)
a_estimated = np.cos(theta_estimated)**2
print('Quantum result, a_estimated: ', a_estimated)
print("Classical result: ",probability[1])

In [ ]:
print('Test OK: ', abs(probability[1]- a_estimated)< 0.005)

In [ ]:
c = iqpe_object.circuit
%qatdisplay c --svg --depth 0

In order to increase the precision of the angle estimation more classical bits can be used

## 6. Amplitude Estimation with IQPE


In order to simplify **Amplitude Estimation** calculations performed with the `IQPE` (see Section 3)  the `IQPEAE` class (which stands for **I**terative **Q**uantum **P**hase **E**stimation **A**mplitude **Estimation**) was developed. This class addresses the initial complexities outlined in Section 5, enabling a more streamlined approach for using **IQPE** in *Amplitude Estimation* calculations. The `IQPEAE` class is implemented in the module **ae_iterative_quantum_pe** of the *Amplitude Estimation* package within the *QQuantLib* library (**QQuantLib/AE/ae_classical_qpe**).

#### Note:
The design of the `IQPEAE` class follows a similar structure to the `MLAE` class from the **QQuantLib/AE/maximum_likelihood_ae** module.

To create an instance of the `IQPEAE` class, the following inputs are mandatory:

1. `oracle`: A QLM `AbstractGate` or `QRoutine` that implements the oracle required for constructing the Grover operator.
2. `target`: The marked state in its binary representation, specified as a Python list.
3. `index`: A list of qubits that will be affected by the Grover operator.

These inputs ensure that the class is properly configured for executing amplitude estimation tasks efficiently and accurately.

In [ ]:
from QQuantLib.DL.data_loading import load_probability

In [ ]:
#Here we created the mandatory oracle
n = 3
N = 2**n
x = np.arange(N)
probability = x/np.sum(x)
oracle = load_probability(probability)

%qatdisplay oracle --depth 0 --svg

#This will be the target state for grover and the list of qubits affected by Grover operator
target = [0, 0, 1]
index = range(oracle.arity)

Now that we have all the mandatory inputs, we can create the `IQPEAE` object. Following the conventions of the `MLAE` class, additional parameters can be provided through a Python dictionary. In this case, the following keys are available for configuration:


* `cbits_number`: An integer specifying the number of classical bits needed for phase estimation.  
- `shots`: The number of measurement shots to execute (default: 100).
- `qpu`: The QPU solver to be used.
- `mcz_qlm`: A boolean flag indicating whether to use the QLM multi-controlled Z gate (`True`, default) or the multiplexor implementation (`False`).

These parameters allow users to customize the behavior of the `IQPEAE` class according to their specific requirements and computational resources.

In [ ]:
from QQuantLib.AE.ae_iterative_quantum_pe import IQPEAE

In [ ]:
ae_iqpe_dict = {
    'qpu': linalg_qpu,
    'cbits_number': 4,
    'shots': 10,
    'mcz_qlm': True      
}

ae_iqpe = IQPEAE(
    oracle=oracle,
    target=target,
    index=index, 
    **ae_iqpe_dict
)

When instantiated the `IQPEAE` the *Grover* operator associated to the oracle operator is created. It can be access using attribute `_grover_oracle`.

In [ ]:
grover_circ = ae_iqpe._grover_oracle
%qatdisplay grover_circ --depth --svg

Now, the `ae_iqpe` object has all the mandatory inputs required to initialize and execute the `IQPE` class. The `run` method performs this step. 

Additionally, when the `run` method is executed, the following properties of the `IQPEAE` class are populated:

- `iqpe_object`: This property is an instance of the `IQPE` class, initialized with the corresponding arguments created by the `IQPEAE` class.
- `final_results`: These are the results obtained from the `iqpe` method of the `iqpe_object` property (see Section 2.3).
- `theta`: The estimated $\theta$ value obtained from the **IQPE** algorithm.
- `ae`: The estimated solution of the *Amplitude Estimation* problem using the **IQPE** algorithm, given by $ a = \cos^2(\theta) $.
- `run_time`: The elapsed time for the complete execution of the `run`  method.

In [ ]:
a_estimated = ae_iqpe.run()

In [ ]:
print('a_estimated = ',a_estimated) 
print('ae_iqpe.final_results= ', ae_iqpe.final_results)
print('ae_iqpe.theta= ', ae_iqpe.theta)
print('ae_iqpe.a= ', ae_iqpe.ae)
print("Classical result: ",probability[1])

In [ ]:
print('Test OK: ',np.abs(ae_iqpe.ae-probability[1])< 0.005)

In [ ]:
print("Elapsed time of a run method: ", ae_iqpe.run_time)

Of course the configuration of the algorithm can be changed by setting the attribute of the class an calling the *run* method again!!

In [ ]:
ae_iqpe.cbits_number = 10
print(ae_iqpe.cbits_number)

In [ ]:
a_estimated = ae_iqpe.run()

In [ ]:
print('a_estimated = ',a_estimated) 
print('ae_pe_withQFT.theta= ', ae_iqpe.theta)
print('ae_pe_withQFT.a= ', ae_iqpe.ae)
print("Classical result: ",probability[1])

In [ ]:
print("Elapsed time of a run method: ", ae_iqpe.run_time)

The `run` method of the `IQPEAE` class creates and configures an object from the `IQPE` class. This object is stored in the `pe_iqpe` attribute of the `IQPEAE` class. All attributes and methods of the `IQPE` class can be accessed through this `pe_iqpe` attribute. For example, you can retrieve the complete quantum circuit that was executed!

As explained earlier, the `iqpe_object` attribute of the `IQPEAE` class is an instance of the `IQPE` class. Therefore, all methods and attributes of the `IQPE` class can be accessed via the **iqpe_object** attribute, although this is **not recommended**.

You can also access the quantum circuit used by the **IQPE** algorithm through these attributes.

In [ ]:
#First we access to the atribute tha stores the IterativeQuantumPE object
attribute_ae_iqpe = ae_iqpe.iqpe_object

#Then we take the circuit attribute from the PhaseEstimationwQFT object (see above in the notebook)
c=attribute_ae_iqpe.circuit
%qatdisplay c --svg --depth 0

When the `run` method is executed following class attributes are populated:

* `circuit_statistics`: python dictionary with the complete statistical information of the circuit.
* `oracle_calls`: number of total oracle calls for a complete execution of the algorithm
* `max_oracle_depth`: maximum number of applications of the oracle for the complete execution of the algorithm.

In [ ]:
ae_iqpe.circuit_statistics

In [ ]:
#Total number of oracle calls
print("The total number of the oracle calls for the IQPEAE was: {}".format(ae_iqpe.oracle_calls))

In [ ]:
#Number of maximum oracle applications
ae_iqpe.max_oracle_depth

In [ ]:
ae_iqpe.quantum_times

In [ ]:
ae_iqpe.quantum_time